<a href="https://colab.research.google.com/github/JLBros10/AI-Science-Fair/blob/main/Stocks_NeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/flatplanet/Pytorch-Tutorial-Youtube/blob/main/simple_NeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [ ]:
class Model(nn.Module):
  def __init__(self, in_features=4, h1=40, h2=100, out_features=1300):
    super().__init__()
    self.fc1 = nn.Linear(in_features, h1)
    self.fc2 = nn.Linear(h1, h2)
    self.out = nn.Linear(h2, out_features)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.out(x)

    return x


In [ ]:
torch.manual_seed(41)
model = Model()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
my_df = pd.read_csv("/content/drive/MyDrive/SF/Datasets for Train n' Test/infolimpioavanzadoTarget.csv")

Mounted at /content/drive


In [ ]:
my_df = my_df.iloc[:,2:11]
my_df

,high,low,close,adjclose,volume,ticker,RSIadjclose15,RSIvolume15,RSIadjclose25
0,18.219000,17.500000,17.760000,17.760000,106600,ASLE,NaN,NaN,NaN
1,18.309999,17.620001,17.660000,17.660000,128700,ASLE,NaN,NaN,NaN
2,17.799999,16.910000,16.950001,16.950001,103100,ASLE,NaN,NaN,NaN
3,16.879999,16.139999,16.170000,16.170000,173600,ASLE,NaN,NaN,NaN
4,16.290001,15.630000,15.710000,15.710000,137800,ASLE,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
7776,23.540001,23.250000,23.290001,22.699928,4900,ATLO,60.782255,47.081752,59.518051
7777,23.610001,23.250000,23.350000,22.758406,9200,ATLO,62.022801,47.747952,60.316762
7778,23.570000,23.219999,23.350000,22.758406,15200,ATLO,62.022801,48.713225,60.316762
7779,23.740000,23.330000,23.610001,23.011820,7100,ATLO,67.186408,47.445460,63.685794


In [ ]:
X = my_df.drop(['close','ticker','RSIadjclose15','RSIvolume15','RSIadjclose25'], axis=1)
y = my_df['close']

In [ ]:
X = X.values
y = y.values

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=41)

In [ ]:
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)

In [ ]:
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
import time

total_hours = 1
seconds_per_hour = 3600
ten_minutes = 60

losses = []

start_time = time.time()
last_logged_time = start_time
end_time = start_time + total_hours * seconds_per_hour

while time.time() < end_time:

    y_pred = model.forward(X_train)

    loss = criterion(y_pred, y_train)


    if time.time() - last_logged_time >= ten_minutes:
        elapsed_time = time.time() - start_time
        print(f'Elapsed Time: {elapsed_time / 3600:.2f} hours, Loss: {loss.detach().numpy()}')
        last_logged_time = time.time()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
print(f'Elapsed Time: {elapsed_time / 3600:.2f} hours, Loss: {loss.detach().numpy()}')

Elapsed Time: 0.02 hours, Loss: 6.745341777801514


In [ ]:
with torch.no_grad():
  y_eval = model.forward(X_test)
  loss = criterion(y_eval, y_test)


In [ ]:
loss

In [ ]:
import time
from functools import total_ordering
correct = 0
total_possible = 0
with torch.no_grad():
  for i, data in enumerate(X_test):
    total_possible += 1
    y_val = model.forward(data)

    #print(f'{i+1}. {str(y_val):38} \t {y_test[i]}')

    if y_val.argmax().item() == y_test[i]:
      correct += 1

print(f'We got {correct}/{total_possible} correct')